In [79]:
import json
import numpy as np
import pandas as pd

In [84]:
with open('data/syntax/train/annotations/train.json') as f:
    syntax_train_data = json.load(f)

with open('data/syntax/val/annotations/val.json') as f:
    syntax_val_data = json.load(f)

with open('data/syntax/test/annotations/test.json') as f:
    syntax_test_data = json.load(f)

syntax_train = pd.json_normalize(syntax_train_data, 'annotations')
categories = pd.json_normalize(syntax_train_data, 'categories')
syntax_val = pd.json_normalize(syntax_val_data, 'annotations')
syntax_test = pd.json_normalize(syntax_test_data, 'annotations')


syntax_train = pd.concat([syntax_train, syntax_val],
                         axis=0).reset_index(drop=True)

In [ ]:
syntax_train.head()

In [70]:
with open('data/stenosis/train/annotations/train.json') as f:
    stenosis_train = json.load(f)

stenosis_train = pd.json_normalize(stenosis_train, 'annotations')
stenosis_train.head()

,id,image_id,category_id,segmentation,area,bbox,iscrowd,attributes.occluded
0,1,676,26,"[[278.0, 291.75, 286.75, 299.25, 289.5, 296.25...",708.0,"[278.0, 245.12, 42.75, 54.13]",0,False
1,2,960,26,"[[294.75, 315.0, 293.0, 322.0, 298.5, 322.5, 3...",457.0,"[293.0, 315.0, 68.0, 23.25]",0,False
2,3,960,26,"[[102.0, 172.25, 109.31, 179.25, 113.0, 175.75...",317.0,"[102.0, 157.38, 34.25, 21.87]",0,False
3,4,960,26,"[[87.69, 192.62, 96.69, 198.75, 95.81, 203.0, ...",388.0,"[84.38, 192.62, 12.31, 43.63]",0,False
4,5,99,26,"[[326.0, 232.62, 330.25, 231.38, 335.0, 239.75...",255.0,"[323.0, 221.25, 25.5, 26.13]",0,False
